In [14]:
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import VotingClassifier
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from sklearn.feature_selection import SelectFpr, SelectFdr, RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

In [15]:

df_ultimate_test = pd.read_table("artificial_test.data", sep=" ", header=None)
df_ultimate_test = df_ultimate_test.drop(df_ultimate_test.columns[500], axis=1) # kolumna 500 do wyrzucenia
df_ultimate_test

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,483,454,513,495,523,469,453,477,506,479,...,455,480,543,259,413,520,485,498,523,510
1,485,508,493,487,478,472,504,476,479,475,...,486,480,535,534,514,452,484,495,548,477
2,483,521,507,475,493,486,421,475,496,483,...,491,476,498,495,508,528,486,465,508,503
3,474,504,576,480,553,483,524,478,483,483,...,521,475,470,463,509,525,479,467,552,517
4,495,474,523,479,495,488,485,476,497,478,...,510,471,522,343,509,520,475,493,506,491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,493,458,503,478,517,479,472,478,444,477,...,469,475,485,443,517,486,474,489,506,506
596,481,484,481,490,449,481,467,478,469,483,...,506,485,508,599,498,527,481,490,455,451
597,485,485,530,480,444,487,462,475,509,494,...,442,474,502,368,453,482,478,481,484,517
598,477,469,528,485,483,469,482,477,494,476,...,473,476,453,638,471,538,470,490,613,492


In [16]:
df_data_X = pd.read_table("artificial_train.data", sep=" ", header=None)
df_data_X = df_data_X.drop(df_data_X.columns[500], axis=1)
df_data_X

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,485,477,537,479,452,471,491,476,475,473,...,477,481,477,485,511,485,481,479,475,496
1,483,458,460,487,587,475,526,479,485,469,...,463,478,487,338,513,486,483,492,510,517
2,487,542,499,468,448,471,442,478,480,477,...,487,481,492,650,506,501,480,489,499,498
3,480,491,510,485,495,472,417,474,502,476,...,491,480,474,572,454,469,475,482,494,461
4,484,502,528,489,466,481,402,478,487,468,...,488,479,452,435,486,508,481,504,495,511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,490,505,503,474,463,461,519,476,518,467,...,467,479,449,588,499,506,475,463,507,501
1996,480,475,476,480,495,482,515,479,480,484,...,464,474,473,424,454,570,476,493,465,485
1997,480,517,631,470,485,474,535,476,493,466,...,501,483,479,687,488,488,483,500,523,481
1998,484,481,505,478,542,477,518,477,510,472,...,487,483,526,750,486,529,484,473,527,485


In [17]:
df_data_Y = pd.read_table("artificial_train.labels", sep=" ", header=None)
df_data_Y

,0
0,-1
1,-1
2,-1
3,1
4,1
...,...
1995,1
1996,-1
1997,-1
1998,1


In [18]:
X_train, X_test, y_train, y_test = train_test_split(df_data_X, df_data_Y, test_size=0.33, random_state=4, stratify=df_data_Y)

In [19]:
# Define the parameter grid






pipe = Pipeline([
    ('reduce_dim', 'passthrough'),
    ('clf', StackingClassifier(
            verbose=True,
            estimators=[
            ('knn1', KNeighborsClassifier()),
            ('knn2', KNeighborsClassifier()),
            ('rf1', RandomForestClassifier()),
            ('rf2', RandomForestClassifier()),
            ('svc1', SVC(probability=True)),
            ('svc2', SVC(probability=True)),
            ('et1', ExtraTreesClassifier()),
            ('et2', ExtraTreesClassifier()),
            ],
            final_estimator= VotingClassifier(
                estimators=[
                    ('f_rf1', RandomForestClassifier()),
                    ('f_svc1', SVC(probability=True)),
                    ('f_et1', ExtraTreesClassifier()),
                    ('f_knn1', KNeighborsClassifier()),
                ]
            )
    ))
])

model_distributions=[
    {
        'clf__knn1__n_neighbors': [3,4,5,6,7],
        'clf__rf1__n_estimators': list(range(100, 501)),
        'clf__rf2__n_estimators': list(range(100, 501)),
        'clf__rf1__max_depth': list(range(5, 13))+ [None],
        'clf__rf2__max_depth': list(range(5, 13))+ [None],
        'clf__svc1__C': uniform(0.1, 10),
        'clf__svc2__C': uniform(0.1, 10),
        'clf__et1__n_estimators': list(range(100, 501)),
        'clf__et2__n_estimators': list(range(100, 501)),
        'clf__et1__max_depth': list(range(5, 13))+ [None],
        'clf__et2__max_depth': list(range(5, 13))+ [None],
        'clf__final_estimator__f_rf1__n_estimators': list(range(100, 501)),
        'clf__final_estimator__f_rf1__max_depth': list(range(5, 13))+ [None],
        'clf__final_estimator__f_svc1__C': uniform(0.1, 10),
        'clf__final_estimator__f_et1__n_estimators': list(range(100, 501)),
        'clf__final_estimator__f_et1__max_depth': list(range(5, 13))+ [None],
        'clf__final_estimator__f_knn1__n_neighbors': [3,4,5,6,7],
    }
]


dimgrid=[
    {
        'reduce_dim': [PCA(svd_solver='full')],
        'reduce_dim__n_components': [4,5,6,7,8,499],
    },
    {
        'reduce_dim': [SelectFpr()],
    },
        {
        'reduce_dim': [SelectFdr()],
    },
]
balanced_accuracy_scorer = make_scorer(balanced_accuracy_score)
param_distributions=[]
for model in model_distributions:
    for dimred in dimgrid:
        param_distributions.append({**dimred, **model})
param_distributions

[{'reduce_dim': [PCA(svd_solver='full')],
  'reduce_dim__n_components': [4, 5, 6, 7, 8, 499],
  'clf__knn1__n_neighbors': [3, 4, 5, 6, 7],
  'clf__rf1__n_estimators': [100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
   108,
   109,
   110,
   111,
   112,
   113,
   114,
   115,
   116,
   117,
   118,
   119,
   120,
   121,
   122,
   123,
   124,
   125,
   126,
   127,
   128,
   129,
   130,
   131,
   132,
   133,
   134,
   135,
   136,
   137,
   138,
   139,
   140,
   141,
   142,
   143,
   144,
   145,
   146,
   147,
   148,
   149,
   150,
   151,
   152,
   153,
   154,
   155,
   156,
   157,
   158,
   159,
   160,
   161,
   162,
   163,
   164,
   165,
   166,
   167,
   168,
   169,
   170,
   171,
   172,
   173,
   174,
   175,
   176,
   177,
   178,
   179,
   180,
   181,
   182,
   183,
   184,
   185,
   186,
   187,
   188,
   189,
   190,
   191,
   192,
   193,
   194,
   195,
   196,
   197,
   198,
   199,
   200,
   201,
   202,
   203,
   

In [20]:
pipe.get_params()


{'memory': None,
 'steps': [('reduce_dim', 'passthrough'),
  ('clf',
   StackingClassifier(estimators=[('knn1', KNeighborsClassifier()),
                                  ('knn2', KNeighborsClassifier()),
                                  ('rf1', RandomForestClassifier()),
                                  ('rf2', RandomForestClassifier()),
                                  ('svc1', SVC(probability=True)),
                                  ('svc2', SVC(probability=True)),
                                  ('et1', ExtraTreesClassifier()),
                                  ('et2', ExtraTreesClassifier())],
                      final_estimator=VotingClassifier(estimators=[('f_rf1',
                                                                    RandomForestClassifier()),
                                                                   ('f_svc1',
                                                                    SVC(probability=True)),
                                              

In [70]:
random_search = RandomizedSearchCV(pipe, param_distributions=param_distributions, n_iter=200, scoring=balanced_accuracy_scorer, cv=3, n_jobs=-1, random_state=1, verbose=4, error_score='raise')
random_search.get_params(deep=True)

{'cv': 3,
 'error_score': 'raise',
 'estimator__memory': None,
 'estimator__steps': [('reduce_dim', 'passthrough'),
  ('clf',
   StackingClassifier(estimators=[('knn1', KNeighborsClassifier()),
                                  ('knn2', KNeighborsClassifier()),
                                  ('rf1', RandomForestClassifier()),
                                  ('rf2', RandomForestClassifier()),
                                  ('svc1', SVC(probability=True)),
                                  ('svc2', SVC(probability=True)),
                                  ('et1', ExtraTreesClassifier()),
                                  ('et2', ExtraTreesClassifier())],
                      final_estimator=ExtraTreesClassifier(), verbose=True))],
 'estimator__verbose': False,
 'estimator__reduce_dim': 'passthrough',
 'estimator__clf': StackingClassifier(estimators=[('knn1', KNeighborsClassifier()),
                                ('knn2', KNeighborsClassifier()),
                               

In [71]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


d:\aml\pd2\venv\lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\aml\pd2\venv\lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=Pipeline(steps=[('reduce_dim', 'passthrough'),
                                             ('clf',
                                              StackingClassifier(estimators=[('knn1',
                                                                              KNeighborsClassifier()),
                                                                             ('knn2',
                                                                              KNeighborsClassifier()),
                                                                             ('rf1',
                                                                              RandomForestClassifier()),
                                                                             ('rf2',
                                                                              RandomForestClassifier()),
                                                                             ('svc1',
                                                                              SVC(probability=True)),
                                                                             ('svc2',
                                                                              SVC(probability=True)),
                                                                             ('et1',
                                                                              ExtraTreesClas...
                                                                    116, 117,
                                                                    118, 119,
                                                                    120, 121,
                                                                    122, 123,
                                                                    124, 125,
                                                                    126, 127,
                                                                    128, 129, ...],
                                         'clf__svc1__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002A5782308E0>,
                                         'clf__svc2__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000002A5781F2AD0>,
                                         'reduce_dim': [SelectFdr()]}],
                   random_state=1, scoring=make_scorer(balanced_accuracy_score),
                   verbose=4)

In [72]:
print(f"best trainig score {random_search.best_score_} ,score test {random_search.score(X_test, y_test)}")

best trainig score 0.8575065396113603 ,score test 0.853030303030303


In [73]:
random_search.best_estimator_.get_params()

{'memory': None,
 'steps': [('reduce_dim', PCA(n_components=5, svd_solver='full')),
  ('clf',
   StackingClassifier(estimators=[('knn1', KNeighborsClassifier(n_neighbors=7)),
                                  ('knn2', KNeighborsClassifier()),
                                  ('rf1',
                                   RandomForestClassifier(n_estimators=140)),
                                  ('rf2',
                                   RandomForestClassifier(n_estimators=210)),
                                  ('svc1',
                                   SVC(C=1.3780699944110364, probability=True)),
                                  ('svc2',
                                   SVC(C=4.167028703340122, probability=True)),
                                  ('et1',
                                   ExtraTreesClassifier(max_depth=6,
                                                        n_estimators=402)),
                                  ('et2',
                                   ExtraT